## 

# Evaluation of LLM on Short Description Classification

In this notebook, we evaluate the performance of a large language model (LLM) on a classification task involving short textual descriptions.

We created a manually labeled test set to assess how well the LLM can handle this limited-context scenario. Since these descriptions are very short, the model may lack sufficient context to perform optimally.


## 1. Test set v3

To ensure the evaluation remains computationally feasible, we created a reduced version of the original dataset (`testset_v1_en_labeled`), resulting in `testset_v3_en_labeled`. This smaller test set includes a maximum of 50 samples per class.

In [ ]:
import json
from collections import defaultdict

def shorten_testset(input_path, output_path, max_per_sdg=50):
    sdg_counts = defaultdict(int)
    selected_entries = []

    with open(input_path, 'r', encoding='utf-8') as infile:
        for line in infile:
            try:
                entry = json.loads(line)
                sdg = entry.get("sdg", "None")
                if sdg_counts[sdg] < max_per_sdg:
                    selected_entries.append(entry)
                    sdg_counts[sdg] += 1
            except json.JSONDecodeError:
                continue  # ignore malformed lines

    with open(output_path, 'w', encoding='utf-8') as outfile:
        for entry in selected_entries:
            json.dump(entry, outfile)
            outfile.write('\n')

shorten_testset("../src/ai/testsets/testset_v1_en_labeled.jsonl", '../src/ai/testsets/testset_v3_en_labeled.jsonl')

In [14]:
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import mannwhitneyu

def tesetset_distribution(testset_path):
    """
    Interactive distribution plot of SDG labels and boxplot of description_number using Plotly.

    Args:
        testset_path (str): Path to the testset file (JSONL format).
    """
    # --- Load data ---
    with open(testset_path, "r") as f:
        data = [json.loads(line) for line in f]

    df = pd.DataFrame(data)

    # --- Plot 1: Interactive Bar Chart of SDG Labels ---
    df_sdg_counts = df["sdg"].value_counts().reset_index()
    df_sdg_counts.columns = ["sdg", "count"]

    fig_bar = px.bar(
        df_sdg_counts,
        x="sdg", y="count",
        title="Distribution of SDG Labels",
        labels={"sdg": "SDG", "count": "Count"},
        color="sdg"
    )
    fig_bar.show()

    # --- Plot 2: Interactive Boxplot of description_number by SDG ---
    fig_box = px.box(
        df[df["sdg"] != "Error"],
        x="sdg",
        y="description_number",
        title="Description Number by SDG Category",
        points="all",
        color="sdg"
    )
    fig_box.update_layout(
        xaxis_title="SDG",
        yaxis_title="Description Number",
        boxmode="group"
    )
    fig_box.show()

    # --- Test statistique: SDG vs None (Mann-Whitney) ---
    df["has_sdg"] = df["sdg"] != "None"
    group1 = df[df["has_sdg"] == False]["description_number"]
    group2 = df[df["has_sdg"] == True]["description_number"]

    stat, p = mannwhitneyu(group1, group2, alternative='two-sided')
    print(f"\nMann-Whitney U Test p-value: {p:.4f}")
    if p < 0.05:
        print("Significant difference in description_number between 'None' and SDG-assigned.")
    else:
        print("No significant difference in description_number between 'None' and SDG-assigned.")

    # --- Plot 3: Interactive Binary Boxplot ---
    fig_binary = px.box(
        df,
        x="has_sdg",
        y="description_number",
        color="has_sdg",
        title="Description Number by SDG Presence",
        points="all",
        labels={"has_sdg": "Has SDG Label", "description_number": "Description Number"}
    )
    fig_binary.update_xaxes(
        tickvals=[True, False],
        ticktext=["SDG Assigned", "None"]
    )
    fig_binary.show()


tesetset_distribution("../src/ai/testsets/testset_v3_en_labeled.jsonl")


Mann-Whitney U Test p-value: 0.0000
Significant difference in description_number between 'None' and SDG-assigned.


## 2. Evaluation fuctions

### 2.1. Regex to extract SDGs from a string

In [4]:
import re

def extract_all_sdgs(text):
    """
    Extracts all SDGs found in a text and returns them as a standardized list.
    
    This function detects multiple SDG references in various formats including:
    - SDG followed by number (e.g., "SDG1", "SDG 2")
    - Numbers with sub-targets (e.g., "16.1", "3.4")
    - Standalone numbers at the beginning or after delimiters
    - Case-insensitive matching
    
    Args:
        text (str): The input text to analyze
        
    Returns:
        list: A list of unique SDGs found in the format ["SDG1", "SDG2", ...], 
            sorted numerically, or empty list if none found
    """
    if not text or not isinstance(text, str):
        return ["None"]
    
    sdg_numbers = set()  # Use set to avoid duplicates
    
    # Pattern 1: SDG followed by number with optional sub-target
    # Captures: SDG1, sdg 2, SDG13.4, etc.
    sdg_pattern = r'(?i)\bsdg\s*(\d{1,2})(?:\.\d+)?\b'
    sdg_matches = re.findall(sdg_pattern, text)
    for match in sdg_matches:
        number = int(match)
        if 1 <= number <= 17:
            sdg_numbers.add(number)
    
    # Pattern 2: Number with sub-target (e.g., "16.1", "3.4")
    # Look for patterns like X.Y where X is 1-17
    number_with_sub_pattern = r'\b(\d{1,2})\.\d+\b'
    sub_matches = re.findall(number_with_sub_pattern, text)
    for match in sub_matches:
        number = int(match)
        if 1 <= number <= 17:
            sdg_numbers.add(number)
    
    # Pattern 3: Standalone numbers at beginning or after delimiters
    # More careful matching to avoid false positives
    # Look for numbers that are:
    # - At the start of the string followed by delimiter or end
    # - After comma/semicolon/colon followed by delimiter or end
    # - After whitespace followed by delimiter or end
    standalone_pattern = r'(?:^|[,;:]\s*|(?<=\s))(\d{1,2})(?=\s*[,;]|\s*$|\s+)'
    standalone_matches = re.findall(standalone_pattern, text.strip())
    for match in standalone_matches:
        number = int(match)
        if 1 <= number <= 17:
            sdg_numbers.add(number)
    
    # Convert to sorted list of formatted strings
    result = [f"SDG{num}" for num in sorted(sdg_numbers)]

    return ["None"] if not result else result


# Test cases to verify functionality
def test_extract_all_sdgs():
    """Test function with various input scenarios"""
    test_cases = [
        # Single SDG cases
        ("SDG1", ["SDG1"]),
        ("sdg1", ["SDG1"]),
        ("SDG 1", ["SDG1"]),
        ("sdg 1", ["SDG1"]),
        ("The target is SDG2.", ["SDG2"]),
        ("Working towards sdg 10 is important.", ["SDG10"]),
        ("... aligned with SDG 17 goals.", ["SDG17"]),
        ("Random text SDG 9 more text", ["SDG9"]),
        ("1", ["SDG1"]),
        ("16.1", ["SDG16"]),
        ("SDG13.4", ["SDG13"]),
        
        # Multiple SDG cases
        ("Something about SDG15 and SDG3.", ["SDG3", "SDG15"]),
        ("3, 4", ["SDG3", "SDG4"]),
        ("SDG1, SDG2, and SDG17", ["SDG1", "SDG2", "SDG17"]),
        ("Working on 1.1, 2.3, and SDG16", ["SDG1", "SDG2", "SDG16"]),
        ("Our project addresses SDG 3, 7.2, and sdg15.", ["SDG3", "SDG7", "SDG15"]),
        ("Goals: 1, 5, 10, 17", ["SDG1", "SDG5", "SDG10", "SDG17"]),
        
        # Edge cases
        ("efzfezf", ["None"]),
        ("", ["None"]),
        ("SDG18", ["None"]),  # Invalid SDG number
        ("SDG0", ["None"]),   # Invalid SDG number
        ("123", ["None"]),    # Number too large
        ("text with no sdgs", ["None"]),
        ("SDG1 and SDG1", ["SDG1"]),  # Duplicates should be removed
    ]
    
    print("Test Results:")
    print("-" * 60)
    all_passed = True
    
    for i, (test_input, expected) in enumerate(test_cases, 1):
        result = extract_all_sdgs(test_input)
        passed = result == expected
        all_passed &= passed
        status = "✓ PASS" if passed else "✗ FAIL"
        
        print(f"{i:2d}. {status} | '{test_input}' -> {result}")
        if not passed:
            print(f"     Expected: {expected}")
    
    print("-" * 60)
    print(f"Overall: {'All tests passed!' if all_passed else 'Some tests failed!'}")
    return all_passed


test_extract_all_sdgs()

Test Results:
------------------------------------------------------------
 1. ✓ PASS | 'SDG1' -> ['SDG1']
 2. ✓ PASS | 'sdg1' -> ['SDG1']
 3. ✓ PASS | 'SDG 1' -> ['SDG1']
 4. ✓ PASS | 'sdg 1' -> ['SDG1']
 5. ✓ PASS | 'The target is SDG2.' -> ['SDG2']
 6. ✓ PASS | 'Working towards sdg 10 is important.' -> ['SDG10']
 7. ✓ PASS | '... aligned with SDG 17 goals.' -> ['SDG17']
 8. ✓ PASS | 'Random text SDG 9 more text' -> ['SDG9']
 9. ✓ PASS | '1' -> ['SDG1']
10. ✓ PASS | '16.1' -> ['SDG16']
11. ✓ PASS | 'SDG13.4' -> ['SDG13']
12. ✓ PASS | 'Something about SDG15 and SDG3.' -> ['SDG3', 'SDG15']
13. ✓ PASS | '3, 4' -> ['SDG3', 'SDG4']
14. ✓ PASS | 'SDG1, SDG2, and SDG17' -> ['SDG1', 'SDG2', 'SDG17']
15. ✓ PASS | 'Working on 1.1, 2.3, and SDG16' -> ['SDG1', 'SDG2', 'SDG16']
16. ✓ PASS | 'Our project addresses SDG 3, 7.2, and sdg15.' -> ['SDG3', 'SDG7', 'SDG15']
17. ✓ PASS | 'Goals: 1, 5, 10, 17' -> ['SDG1', 'SDG5', 'SDG10', 'SDG17']
18. ✓ PASS | 'efzfezf' -> ['None']
19. ✓ PASS | '' -> ['None

True

### 2.2. Générate evaluation file

In [ ]:
import os
import json
import time
from tqdm import tqdm
from datetime import datetime
from ai.models.classify_patent import Classify_patent
from api.config.ai_config import ai_client

def run_evaluation(model: Classify_patent, testset_path: str, output_path: str):
    """
    Runs evaluation on a JSONL test dataset, classifying each description and saving results.

    If the specified output path already exists, appends a numeric suffix (_1, _2, ...) 
    to avoid overwriting the existing file.

    For each line in the test set:
        - Extracts the 'description_text' field
        - Measures the prediction time
        - Classifies the description using `classify_description`
        - Saves the enriched line with classification results and prediction time

    At the end, appends a metadata entry containing:
        - model_name
        - testset_path
        - prompt_template_path (if available)
        - creation date

    Args:
        testset_path (str): Path to the input .jsonl test dataset.
        output_path (str): Desired path to save the output .jsonl file.

    Returns:
        None
    """

    def get_safe_output_path(path):
        """Generates a non-conflicting file path by appending a numeric suffix if needed."""
        base, ext = os.path.splitext(path)
        counter = 1
        while os.path.exists(path):
            path = f"{base}_{counter}{ext}"
            counter += 1
        return path

    # Ensure the output file won't overwrite an existing file
    safe_output_path = get_safe_output_path(output_path)

    # Read all lines for progress tracking
    with open(testset_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    with open(safe_output_path, "w", encoding="utf-8") as out_f:
        for line in tqdm(lines, desc="Running evaluation"):
            data = json.loads(line)
            description_text = data.get("description_text", "")

            # Rename sdg in true_sdg
            if "sdg" in data:
                data["true_sdg"] = data.pop("sdg")

            # Time the classification
            start_time = time.time()
            sdg_balise, reason_balise = model.generate_response(description_text)
            prediction_time = time.time() - start_time

            # Add classification results and timing to the entry
            data.update({
                "sdg_balise": sdg_balise,
                "reason_balise": reason_balise,
                "prediction_time": prediction_time
            })

            # Write enriched data to output
            out_f.write(json.dumps(data) + "\n")

        # Append metadata at the end
        meta_data = {
            "meta_data": {
                "model_name": model.model_name,
                "testset_path": testset_path,
                "prompt_name": getattr(model, "prompt_name", "N/A"),
                "date_creation": datetime.now().isoformat()
            }
        }
        out_f.write(json.dumps(meta_data) + "\n")

### 2.3. Plot evaluation 

In [7]:
from collections import defaultdict

def show_evaluation_mono(evaluation_path: str):
    """
    Evaluates the predictions in a previously generated evaluation .jsonl file.

    Each prediction line must contain:
        - "sdg": the ground truth SDG label (e.g., "SDG3")
        - "sdg_balise": predicted SDG(s) as a string (e.g., "SDG1, SDG3, SDG7")

    A prediction is considered correct if the ground truth SDG is among the predicted SDGs.

    Statistics reported:
        - Total predictions and correct ones
        - Overall accuracy
        - Average and total prediction time
        - SDG-specific accuracy breakdown
        - Sample of incorrect predictions

    Args:
        evaluation_path (str): Path to the JSONL evaluation file created by `run_evaluation`.

    Returns:
        None
    """

    results = []
    meta_data = {}
    total_predictions = 0
    correct_predictions = 0
    total_time = 0.0
    sdg_breakdown = defaultdict(lambda: {"correct": 0, "total": 0})

    # Read and evaluate each line
    with open(evaluation_path, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)
            
            if "meta_data" in data:
                meta_data = data["meta_data"]
                continue  # Skip metadata line
            
            # To avoid empty line
            if "patent_number" in data:
                true_sdg = data.get("true_sdg", "").strip()
                predicted_sdg_raw = data.get("sdg_balise", "")
                predicted_sdg_list = extract_all_sdgs(predicted_sdg_raw)

                is_correct = true_sdg in predicted_sdg_list
                total_predictions += 1
                if is_correct:
                    correct_predictions += 1

                prediction_time = data.get("prediction_time", 0.0)
                total_time += prediction_time

                sdg_breakdown[true_sdg]["total"] += 1
                if is_correct:
                    sdg_breakdown[true_sdg]["correct"] += 1

                results.append({
                    "patent_number": data.get("patent_number", "N/A"),
                    "description_text": data.get("description_text", ""),
                    "true_sdg": true_sdg,
                    "predicted_sdg": predicted_sdg_list,
                    "is_correct": is_correct
                })

    # Compute summary statistics
    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0.0
    avg_prediction_time = total_time / total_predictions if total_predictions > 0 else 0.0
    predictions_per_second = total_predictions / total_time if total_time > 0 else 0.0
    incorrect_predictions_count = total_predictions - correct_predictions

    # Print metadata
    print(f"\n=== META DATA ===")
    for key, value in meta_data.items():
        print(f"{key}: {value}")

    # Print summary
    print(f"\n=== EVALUATION RESULTS ===")
    print(f"Total predictions: {total_predictions}")
    print(f"Correct predictions: {correct_predictions}")
    print(f"Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"Total time: {total_time:.2f} seconds")
    print(f"Average prediction time: {avg_prediction_time:.4f} seconds")
    print(f"Predictions per second: {predictions_per_second:.2f}")
    print(f"Incorrect predictions: {incorrect_predictions_count}")
    print(f"Results loaded from: {evaluation_path}")

    # Show SDG breakdown
    print(f"\n=== SDG BREAKDOWN ===")
    for sdg, stats in sorted(sdg_breakdown.items()):
        sdg_accuracy = stats["correct"] / stats["total"] if stats["total"] > 0 else 0.0
        print(f"{sdg}: {stats['correct']}/{stats['total']} ({sdg_accuracy:.2%})")

    # Show some examples of incorrect predictions
    incorrect_results = [r for r in results if not r["is_correct"]]
    if incorrect_results:
        print(f"\n=== SAMPLE INCORRECT PREDICTIONS ===")
        for i, result in enumerate(incorrect_results[:5]):
            print(f"\nExample {i+1}:")
            print(f"Patent: {result['patent_number']}")
            print(f"Description: {result['description_text'][:100]}...")
            print(f"True SDG: {result['true_sdg']}")
            print(f"Predicted SDG: {result['predicted_sdg']}")

## 3. Reproducibility

We first examine whether running the same experiment twice times yields consistent results.

### 3.1. Generate evaluation files

In [ ]:
# Initialize model
model = Classify_patent(client=ai_client, model_name="qwen3:4b", prompt_name="sdg_label_prompt.md")

In [ ]:
# Generate evaluations file 1
run_evaluation(model=model, testset_path="../src/ai/testsets/testset_v3_en_labeled.jsonl", 
               output_path="../src/ai/evaluations/llm_reproductibility_4b.jsonl")

Running evaluation:   0%|          | 0/328 [00:00<?, ?it/s]

In [ ]:
# Generate evaluations file 2
run_evaluation(model=model, testset_path="../src/ai/testsets/testset_v3_en_labeled.jsonl", 
               output_path="../src/ai/evaluations/llm_reproductibility_4b_1.jsonl")

### 3.2. Plot evaluation

In [8]:
show_evaluation_mono("../src/ai/evaluations/llm_reproductibility_4b.jsonl")


=== META DATA ===
model_name: qwen3:4b
testset_path: ../src/ai/testsets/testset_v3_en_labeled.jsonl
prompt_template_path: sdg_label_prompt.md
date_creation: 2025-05-24T21:49:09.555008

=== EVALUATION RESULTS ===
Total predictions: 328
Correct predictions: 150
Accuracy: 0.4573 (45.73%)
Total time: 5450.24 seconds
Average prediction time: 16.6166 seconds
Predictions per second: 0.06
Incorrect predictions: 178
Results loaded from: ../src/ai/evaluations/llm_reproductibility_4b.jsonl

=== SDG BREAKDOWN ===
None: 5/50 (10.00%)
SDG10: 0/12 (0.00%)
SDG11: 3/18 (16.67%)
SDG12: 11/50 (22.00%)
SDG13: 13/18 (72.22%)
SDG14: 5/5 (100.00%)
SDG15: 2/4 (50.00%)
SDG16: 2/2 (100.00%)
SDG2: 0/3 (0.00%)
SDG3: 47/50 (94.00%)
SDG4: 1/7 (14.29%)
SDG6: 21/27 (77.78%)
SDG7: 9/32 (28.12%)
SDG9: 31/50 (62.00%)

=== SAMPLE INCORRECT PREDICTIONS ===

Example 1:
Patent: EP3827727A1
Description: In the first embodiment, the first mop 1a and the second mop 1 b are substantially triangular-like....
True SDG: None
Pred

In [24]:
show_evaluation_mono("../src/ai/evaluations/llm_reproductibility_4b_1.jsonl")


=== META DATA ===
model_name: qwen3:4b
testset_path: ../src/ai/testsets/testset_v3_en_labeled.jsonl
prompt_template_path: sdg_label_prompt.md
date_creation: 2025-05-24T22:54:58.027378

=== EVALUATION RESULTS ===
Total predictions: 328
Correct predictions: 156
Accuracy: 0.4756 (47.56%)
Total time: 3948.20 seconds
Average prediction time: 12.0372 seconds
Predictions per second: 0.08
Incorrect predictions: 172
Results loaded from: ../src/ai/evaluations/llm_reproductibility_4b_1.jsonl

=== SDG BREAKDOWN ===
None: 9/50 (18.00%)
SDG10: 0/12 (0.00%)
SDG11: 4/18 (22.22%)
SDG12: 11/50 (22.00%)
SDG13: 13/18 (72.22%)
SDG14: 5/5 (100.00%)
SDG15: 2/4 (50.00%)
SDG16: 2/2 (100.00%)
SDG2: 0/3 (0.00%)
SDG3: 48/50 (96.00%)
SDG4: 2/7 (28.57%)
SDG6: 20/27 (74.07%)
SDG7: 10/32 (31.25%)
SDG9: 30/50 (60.00%)

=== SAMPLE INCORRECT PREDICTIONS ===

Example 1:
Patent: EP4287161A1
Description: According to the driving assistance system 100 of the embodiment described above, the following effe...
True SDG: SDG11

## 4. Model Size

We tested various model sizes and selected the Qwen3 model for evaluation. Specifically, we assessed the performance of three variants: 1.7B, 4B, 8B and 14B parameters.


### 4.1. Generate evaluation files

In [ ]:
model = Classify_patent(client=ai_client, model_name="qwen3:1.7b", prompt_name="sdg_label_prompt.md")
run_evaluation(model=model, testset_path="../src/ai/testsets/testset_v3_en_labeled.jsonl", 
               output_path="../src/ai/evaluations/llm_model_size_qwen3_1-7b.jsonl")

In [ ]:
model = Classify_patent(client=ai_client, model_name="qwen3:4b", prompt_name="sdg_label_prompt.md")
run_evaluation(model=model, testset_path="../src/ai/testsets/testset_v3_en_labeled.jsonl", 
               output_path="../src/ai/evaluations/llm_model_size_qwen3_4b.jsonl")

In [ ]:
model = Classify_patent(client=ai_client, model_name="qwen3:8b", prompt_name="sdg_label_prompt.md")
run_evaluation(model=model, testset_path="../src/ai/testsets/testset_v3_en_labeled.jsonl", 
               output_path="../src/ai/evaluations/llm_model_size_qwen3_8b.jsonl")

In [ ]:
model = Classify_patent(client=ai_client, model_name="qwen3:14b", prompt_name="sdg_label_prompt.md")
run_evaluation(model=model, testset_path="../src/ai/testsets/testset_v3_en_labeled.jsonl", 
               output_path="../src/ai/evaluations/llm_model_size_qwen3_14b.jsonl")

### 4.2. Plot evaluation

In [ ]:
# The model 1.7b has some empty <sdg> balises wich is a problem.
show_evaluation_mono("../src/ai/evaluations/llm_model_size_qwen3_1-7b.jsonl")


=== META DATA ===

=== EVALUATION RESULTS ===
Total predictions: 328
Correct predictions: 140
Accuracy: 0.4268 (42.68%)
Total time: 3089.17 seconds
Average prediction time: 9.4182 seconds
Predictions per second: 0.11
Incorrect predictions: 188
Results loaded from: ../src/ai/evaluations/llm_model_size_qwen3_1-7b.jsonl

=== SDG BREAKDOWN ===
None: 14/50 (28.00%)
SDG10: 0/12 (0.00%)
SDG11: 6/18 (33.33%)
SDG12: 19/50 (38.00%)
SDG13: 13/18 (72.22%)
SDG14: 3/5 (60.00%)
SDG15: 1/4 (25.00%)
SDG16: 1/2 (50.00%)
SDG2: 0/3 (0.00%)
SDG3: 46/50 (92.00%)
SDG4: 2/7 (28.57%)
SDG6: 19/27 (70.37%)
SDG7: 4/32 (12.50%)
SDG9: 12/50 (24.00%)

=== SAMPLE INCORRECT PREDICTIONS ===

Example 1:
Patent: EP4516757A2
Description: A supplementary cementitious material (SCM) is a material used in conjunction with or as a partial r...
True SDG: SDG9
Predicted SDG: ['SDG11']

Example 2:
Patent: EP4054160A1
Description: Further, the switch 128L may be a pressure sensitive or membrane switch defined as a fingertip-size

In [ ]:
show_evaluation_mono("../src/ai/evaluations/llm_model_size_qwen3_4b.jsonl")


=== META DATA ===

=== EVALUATION RESULTS ===
Total predictions: 328
Correct predictions: 149
Accuracy: 0.4543 (45.43%)
Total time: 3888.93 seconds
Average prediction time: 11.8565 seconds
Predictions per second: 0.08
Incorrect predictions: 179
Results loaded from: ../src/ai/evaluations/llm_model_size_qwen3_4b.jsonl

=== SDG BREAKDOWN ===
None: 5/50 (10.00%)
SDG10: 0/12 (0.00%)
SDG11: 5/18 (27.78%)
SDG12: 8/50 (16.00%)
SDG13: 12/18 (66.67%)
SDG14: 4/5 (80.00%)
SDG15: 1/4 (25.00%)
SDG16: 2/2 (100.00%)
SDG2: 0/3 (0.00%)
SDG3: 47/50 (94.00%)
SDG4: 1/7 (14.29%)
SDG6: 21/27 (77.78%)
SDG7: 11/32 (34.38%)
SDG9: 32/50 (64.00%)

=== SAMPLE INCORRECT PREDICTIONS ===

Example 1:
Patent: EP4516757A2
Description: A supplementary cementitious material (SCM) is a material used in conjunction with or as a partial r...
True SDG: SDG9
Predicted SDG: ['SDG12', 'SDG13']

Example 2:
Patent: EP3827727A1
Description: In the first embodiment, the first mop 1a and the second mop 1 b are substantially triangul

In [ ]:
show_evaluation_mono("../src/ai/evaluations/llm_model_size_qwen3_8b.jsonl")


=== META DATA ===

=== EVALUATION RESULTS ===
Total predictions: 328
Correct predictions: 162
Accuracy: 0.4939 (49.39%)
Total time: 6087.96 seconds
Average prediction time: 18.5609 seconds
Predictions per second: 0.05
Incorrect predictions: 166
Results loaded from: ../src/ai/evaluations/llm_model_size_qwen3_8b.jsonl

=== SDG BREAKDOWN ===
None: 24/50 (48.00%)
SDG10: 0/12 (0.00%)
SDG11: 4/18 (22.22%)
SDG12: 6/50 (12.00%)
SDG13: 10/18 (55.56%)
SDG14: 4/5 (80.00%)
SDG15: 1/4 (25.00%)
SDG16: 2/2 (100.00%)
SDG2: 0/3 (0.00%)
SDG3: 42/50 (84.00%)
SDG4: 2/7 (28.57%)
SDG6: 22/27 (81.48%)
SDG7: 16/32 (50.00%)
SDG9: 29/50 (58.00%)

=== SAMPLE INCORRECT PREDICTIONS ===

Example 1:
Patent: EP4516757A2
Description: A supplementary cementitious material (SCM) is a material used in conjunction with or as a partial r...
True SDG: SDG9
Predicted SDG: ['SDG13']

Example 2:
Patent: EP4287161A1
Description: According to the driving assistance system 100 of the embodiment described above, the following eff

In [ ]:
show_evaluation_mono("../src/ai/evaluations/llm_model_size_qwen3_14b.jsonl")


=== META DATA ===
model_name: qwen3:14b
testset_path: ../src/ai/testsets/testset_v3_en_labeled.jsonl
prompt_template_path: sdg_label_prompt.md
date_creation: 2025-05-25T17:18:30.614956

=== EVALUATION RESULTS ===
Total predictions: 328
Correct predictions: 150
Accuracy: 0.4573 (45.73%)
Total time: 9016.78 seconds
Average prediction time: 27.4902 seconds
Predictions per second: 0.04
Incorrect predictions: 178
Results loaded from: ../src/ai/evaluations/llm_model_size_qwen3_14b.jsonl

=== SDG BREAKDOWN ===
None: 37/50 (74.00%)
SDG10: 0/12 (0.00%)
SDG11: 5/18 (27.78%)
SDG12: 5/50 (10.00%)
SDG13: 12/18 (66.67%)
SDG14: 4/5 (80.00%)
SDG15: 1/4 (25.00%)
SDG16: 1/2 (50.00%)
SDG2: 0/3 (0.00%)
SDG3: 41/50 (82.00%)
SDG4: 1/7 (14.29%)
SDG6: 12/27 (44.44%)
SDG7: 5/32 (15.62%)
SDG9: 26/50 (52.00%)

=== SAMPLE INCORRECT PREDICTIONS ===

Example 1:
Patent: EP4516757A2
Description: A supplementary cementitious material (SCM) is a material used in conjunction with or as a partial r...
True SDG: SDG9
Pre

## 5. Shorter model Prompt

### 5.1. Generate evaluation files

In [ ]:
model = Classify_patent(client=ai_client, model_name="qwen3:4b", prompt_name="sdg_label_prompt_short.md")
run_evaluation(model=model, testset_path="../src/ai/testsets/testset_v3_en_labeled.jsonl", 
               output_path="../src/ai/evaluations/llm_prompt_short_4b.jsonl")

In [ ]:
model = Classify_patent(client=ai_client, model_name="qwen3:14b", prompt_name="sdg_label_prompt_short.md")
run_evaluation(model=model, testset_path="../src/ai/testsets/testset_v3_en_labeled.jsonl", 
               output_path="../src/ai/evaluations/llm_prompt_short_14b.jsonl")

### 5.2. Plot evaluation

In [ ]:
show_evaluation_mono("../src/ai/evaluations/llm_prompt_short_4b.jsonl")


=== META DATA ===
model_name: qwen3:4b
testset_path: ../src/ai/testsets/testset_v3_en_labeled.jsonl
prompt_template_path: sdg_label_prompt_short.md
date_creation: 2025-05-24T23:57:21.573880

=== EVALUATION RESULTS ===
Total predictions: 328
Correct predictions: 148
Accuracy: 0.4512 (45.12%)
Total time: 3743.36 seconds
Average prediction time: 11.4127 seconds
Predictions per second: 0.09
Incorrect predictions: 180
Results loaded from: ../src/ai/evaluations/llm_prompt_short_4b.jsonl

=== SDG BREAKDOWN ===
None: 26/50 (52.00%)
SDG10: 0/12 (0.00%)
SDG11: 3/18 (16.67%)
SDG12: 9/50 (18.00%)
SDG13: 14/18 (77.78%)
SDG14: 4/5 (80.00%)
SDG15: 1/4 (25.00%)
SDG16: 0/2 (0.00%)
SDG2: 1/3 (33.33%)
SDG3: 33/50 (66.00%)
SDG4: 2/7 (28.57%)
SDG6: 15/27 (55.56%)
SDG7: 11/32 (34.38%)
SDG9: 29/50 (58.00%)

=== SAMPLE INCORRECT PREDICTIONS ===

Example 1:
Patent: EP4054160A1
Description: Further, the switch 128L may be a pressure sensitive or membrane switch defined as a fingertip-sized...
True SDG: SDG16
P

In [ ]:
show_evaluation_mono("../src/ai/evaluations/llm_prompt_short_14b.jsonl")


=== META DATA ===
model_name: qwen3:14b
testset_path: ../src/ai/testsets/testset_v3_en_labeled.jsonl
prompt_template_path: sdg_label_prompt_short.md
date_creation: 2025-05-25T06:50:25.656681

=== EVALUATION RESULTS ===
Total predictions: 328
Correct predictions: 168
Accuracy: 0.5122 (51.22%)
Total time: 7865.16 seconds
Average prediction time: 23.9791 seconds
Predictions per second: 0.04
Incorrect predictions: 160
Results loaded from: ../src/ai/evaluations/llm_prompt_short_14b.jsonl

=== SDG BREAKDOWN ===
None: 33/50 (66.00%)
SDG10: 0/12 (0.00%)
SDG11: 2/18 (11.11%)
SDG12: 8/50 (16.00%)
SDG13: 14/18 (77.78%)
SDG14: 3/5 (60.00%)
SDG15: 2/4 (50.00%)
SDG16: 1/2 (50.00%)
SDG2: 0/3 (0.00%)
SDG3: 38/50 (76.00%)
SDG4: 2/7 (28.57%)
SDG6: 15/27 (55.56%)
SDG7: 7/32 (21.88%)
SDG9: 43/50 (86.00%)

=== SAMPLE INCORRECT PREDICTIONS ===

Example 1:
Patent: EP4054160A1
Description: Further, the switch 128L may be a pressure sensitive or membrane switch defined as a fingertip-sized...
True SDG: SDG16
